In [2]:
import sqlite3
import pandas as pd

# 连接到 SQLite 数据库
db_path = "../anime_data.db"  # 替换为你的 SQLite 数据库文件路径
conn = sqlite3.connect(db_path)

In [3]:
# 查询数据库中的所有表名
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql(query, conn)
    return tables['name'].tolist()

table_names = get_table_names(conn)
print("Tables in database:", table_names)


Tables in database: ['Anime', 'Category', 'sqlite_sequence', 'Anime_Category', 'Production', 'Anime_Production', 'Comments']


In [4]:
# 将所有表导入 Pandas 数据帧
def load_tables_to_pandas(conn, table_names):
    dataframes = {}
    for table in table_names:
        dataframes[table] = pd.read_sql(f"SELECT * FROM {table};", conn)
    return dataframes

dataframes = load_tables_to_pandas(conn, table_names)


In [5]:
# 查看某张表的数据
for table, df in dataframes.items():
    print(f"Table: {table}")
    print(df.head())  # 查看前5行数据
    print("\n")


Table: Anime
   anime_id                           anime_title score  score_count  \
0    208827                         少女与战车 最终章 第4话   7.8         1150   
1    217021                                    冥王   7.3         2994   
2    228715                             你想活出怎样的人生   7.2         4916   
3    235920                            超级马力欧兄弟大电影   7.1         1281   
4    244930  Princess Principal Crown Handler 第3章   6.7          504   

  release_date  
0   2023-10-06  
1   2023-10-26  
2   2023-07-14  
3   2023-04-05  
4   2023-04-07  


Table: Category
   category_id category_name
0            1            原创
1            2            欧美
2            3            短片
3            4          2023
4            5          动画短片


Table: sqlite_sequence
         name    seq
0    Category   1587
1  Production    240
2    Comments  58644


Table: Anime_Category
   anime_id  category_id
0    448689            1
1    448689            2
2    448689            3
3    448689            4
4 

In [6]:
conn.close()